In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.naive_bayes import GaussianNB
from collections import Counter
import plotly.graph_objs as go
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from IPython.display import display
from numpy.random import RandomState
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
sns.set(style='white', context='notebook', palette='deep')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## BLM19432E - Introdiction to Data Science

#### Classifier Evaluation Techniques

Use the same audit data, but this time, use the original dataset which is attached below. 

Previously, you have used only the "accuracy" measure to evaluate your classifier. Now, you need to evaluate your classifier with respect to several measures.

Your homework is to:

1. Do research about the other classifier evaluation measures.
2. Repeat the 5-fold CV (using the whole dataset) for each of those measures. 
3. Do research about the methods that also help to visually assess the classifiers. 


# Read data

In [ ]:
# Loading the dataset
df= pd.read_csv(r'../input/all-audit-data/audit_risk_original.csv')

In [ ]:
#df= pd.read_csv(r'/kaggle/input/test-data/test.csv')

In [ ]:
df.head()

## Data description

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

**Missing Values**

## See if there are any NaNs in the dataframe

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = ((df.isnull().sum()/df.isnull().count()).sort_values(ascending=False))* 100
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
plt.xticks(rotation='90')
sns.barplot(x=percent.index[:30], y=percent[:30])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
df['Money_Value'].fillna(df['Money_Value'].value_counts().iloc[0], inplace=True)

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = ((df.isnull().sum()/df.isnull().count()).sort_values(ascending=False))* 100
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
f, ax = plt.subplots(figsize=(10, 8))
plt.xticks(rotation='90')
sns.barplot(x=percent.index[:30], y=percent[:30])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
lbl_train = LabelEncoder() 
lbl_train.fit(list(df['LOCATION_ID'].values)) # take the values of columns
df['LOCATION_ID'] = lbl_train.transform(list(df['LOCATION_ID'].values)) # classify the similar value under same number 

**Logistic Regression**

**Support Vector Machines**

**Naive Bayes**

**Decision Trees**

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['knn'] = KNeighborsClassifier()
    models['cart'] = DecisionTreeClassifier()
    models['svm'] = SVC()
    models['NN']=MLPClassifier()
    models['RF']=RandomForestClassifier()
    return models

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score

kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
X=df.drop('Risk',axis='columns')
y=df['Risk']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# Compare Algorithms
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('DTree', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    cv_results = cross_val_score(model,X,y, cv=kfolds, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
# Modeling step Test differents algorithms 
classifiers = []
classifiers.append(LogisticRegression())
classifiers.append(SVC())
classifiers.append(GaussianNB())
classifiers.append(DecisionTreeClassifier())

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier,X, y , scoring = "accuracy", cv=kfolds,n_jobs= 4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["LogisticRegression","SVC","GaussianNB","DecisionTreeClassifier"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

[Tree🎄 ](https://towardsdatascience.com/visualizing-decision-trees-with-python-scikit-learn-graphviz-matplotlib-1c50b4aa68dc)

In [ ]:
from sklearn.metrics import classification_report
import graphviz
from sklearn import tree

model = DecisionTreeClassifier()
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kfolds)
print ("Cross-validation (mean) accuracy-score: %.5f\n" % np.mean(scores))
#print('cross-validation classification report ')
#classification_report(y, model.predict(X))

model.fit(X_train, y_train)
predicted = model.predict(X_test)
print('cross-validation classification report ')
print(classification_report(y_test, predicted))
print("Test accuracy score (with trained model): %.5f" % model.score(X_test, y_test))
cm = confusion_matrix(y_test, predicted)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
model = SVC()
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kfolds)
print ("Cross-validation (mean) accuracy-score: %.5f\n" % np.mean(scores))
#print('cross-validation classification report ')
#classification_report(y, model.predict(X))

model.fit(X_train, y_train)
predicted = model.predict(X_test)
print('cross-validation classification report ')
print(classification_report(y_test, predicted))
print("Test accuracy score (with trained model): %.5f" % model.score(X_test, y_test))
cm = confusion_matrix(y_test, predicted)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
model = LogisticRegression()
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kfolds)
print ("Cross-validation (mean) accuracy-score: %.5f\n" % np.mean(scores))
#print('cross-validation classification report ')
#classification_report(y, model.predict(X))

model.fit(X_train, y_train)
predicted = model.predict(X_test)
print('cross-validation classification report ')
print(classification_report(y_test, predicted))
print("Test accuracy score (with trained model): %.5f" % model.score(X_test, y_test))
cm = confusion_matrix(y_test, predicted)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
model = GaussianNB()
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kfolds)
print ("Cross-validation (mean) accuracy-score: %.5f\n" % np.mean(scores))
#print('cross-validation classification report ')
#classification_report(y, model.predict(X))

model.fit(X_train, y_train)
predicted = model.predict(X_test)
print('cross-validation classification report ')
print(classification_report(y_test, predicted))
print("Test accuracy score (with trained model): %.5f" % model.score(X_test, y_test))
cm = confusion_matrix(y_test, predicted)
cm_display = ConfusionMatrixDisplay(cm).plot()

### ROC 

https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py

In [ ]:
models = []
models.append(('Logistic Regression',LogisticRegression()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('SVM', SVC()))
models.append(('GaussianNB', GaussianNB()))


# Run classifier with cross-validation and plot ROC curves
for name, model in models:
    cv = StratifiedKFold(n_splits=5)
    classifier =model

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    fig=plt.figure(figsize=[10,10])
    ax = fig.add_subplot(111)
    for i, (train, test) in enumerate(cv.split(X, y)):
        classifier.fit(X_train, y_train)
        viz = plot_roc_curve(classifier, X_test, y_test,
                            name='ROC fold {}'.format(i),
                            alpha=0.3, lw=1, ax=ax)
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
          title=name+": Receiver operating characteristic example")
    ax.legend(loc="lower right")
    plt.show()

# Reference 
https://github.com/justmarkham/DAT8/blob/master/other/model_evaluation_comparison.md

https://github.com/justmarkham/scikit-learn-videos/blob/master/09_classification_metrics.ipynb